In [ ]:
import pandas as pd
import numpy as np
import python_functions as pf
import warnings
warnings.filterwarnings("ignore")

In [ ]:
PISA_m = pd.read_excel('data/2000-2022_m_r.xls', sheet_name='mathematics', header=11)
PISA_r = pd.read_excel('data/2000-2022_m_r.xls', sheet_name='reading', header=11)
PISA_sc = pd.read_excel('data/2000-2022_sc.xls', sheet_name='science', header=11)

In [ ]:
PISA_m = PISA_m.drop('Unnamed: 0', axis=1)
PISA_r = PISA_r.drop('Unnamed: 0', axis=1)
PISA_sc = PISA_sc.drop('Unnamed: 0', axis=1)

---

Put column names in lower case, snake case and rename:

In [ ]:
dataframes = [PISA_m, PISA_r, PISA_sc]

pf.columns_lower_snake_case(dataframes)

In [ ]:
PISA_m.sample(5)

In [ ]:
PISA_r.sample(5)

In [ ]:
PISA_sc.sample(5)

---

Create new DataFrames, for each subject and year one DataFrame:

In [ ]:
years = ['2000', '2003', '2006', '2009', '2012', '2015', '2018', '2022']

# Create a new list to safe the new DataFrames
new_dataframes = []

# Iterate over every year
for year in years:
    # Filter the row for every DataFrame for each year
    PISA_m_year = PISA_m[PISA_m['year'] == year].copy()
    PISA_r_year = PISA_r[PISA_r['year'] == year].copy()
    PISA_sc_year = PISA_sc[PISA_sc['year'] == year].copy()
    
    # Add the new DataFrames to the list
    new_dataframes.extend([PISA_m_year, PISA_r_year, PISA_sc_year])

# Save the newly created DataFrames in new Variables
PISA_m_2000, PISA_r_2000, PISA_sc_2000 = new_dataframes[:3]
PISA_m_2003, PISA_r_2003, PISA_sc_2003 = new_dataframes[3:6]
PISA_m_2006, PISA_r_2006, PISA_sc_2006 = new_dataframes[6:9]
PISA_m_2009, PISA_r_2009, PISA_sc_2009 = new_dataframes[9:12]
PISA_m_2012, PISA_r_2012, PISA_sc_2012 = new_dataframes[12:15]
PISA_m_2015, PISA_r_2015, PISA_sc_2015 = new_dataframes[15:18]
PISA_m_2018, PISA_r_2018, PISA_sc_2018 = new_dataframes[18:21]
PISA_m_2022, PISA_r_2022, PISA_sc_2022 = new_dataframes[21:]

In [ ]:
dataframes = [PISA_m_2000, PISA_r_2000, PISA_sc_2000,
              PISA_m_2003, PISA_r_2003, PISA_sc_2003,
              PISA_m_2006, PISA_r_2006, PISA_sc_2006,
              PISA_m_2009, PISA_r_2009, PISA_sc_2009,
              PISA_m_2012, PISA_r_2012, PISA_sc_2012,
              PISA_m_2015, PISA_r_2015, PISA_sc_2015,
              PISA_m_2018, PISA_r_2018, PISA_sc_2018,
              PISA_m_2022, PISA_r_2022, PISA_sc_2022]

for df in dataframes:
    df.reset_index(inplace=True, drop=True)

Check that only one year is saved in each DataFrame:

In [ ]:
print(PISA_m_2000['year'].unique(), 
      PISA_m_2003['year'].unique(), 
      PISA_m_2006['year'].unique(), 
      PISA_m_2009['year'].unique(), 
      PISA_m_2012['year'].unique(), 
      PISA_m_2015['year'].unique(), 
      PISA_m_2018['year'].unique(), 
      PISA_m_2022['year'].unique())

In [ ]:
print(PISA_r_2000['year'].unique(),
    PISA_r_2003['year'].unique(),
    PISA_r_2006['year'].unique(),
    PISA_r_2009['year'].unique(),
    PISA_r_2012['year'].unique(),
    PISA_r_2015['year'].unique(),
    PISA_r_2018['year'].unique(),
    PISA_r_2022['year'].unique())

In [ ]:
print(PISA_sc_2000['year'].unique(),
    PISA_sc_2003['year'].unique(),
    PISA_sc_2006['year'].unique(),
    PISA_sc_2009['year'].unique(),
    PISA_sc_2012['year'].unique(),
    PISA_sc_2015['year'].unique(),
    PISA_sc_2018['year'].unique(),
    PISA_sc_2022['year'].unique())

---

Change the types for two columns and round the numbers:

In [ ]:
PISA_m_2006.head(10)

In [ ]:
# dataframes = [PISA_m_2000, PISA_r_2000, PISA_sc_2000,
#               PISA_m_2003, PISA_r_2003, PISA_sc_2003,
#               PISA_m_2006, PISA_r_2006, PISA_sc_2006,
#               PISA_m_2009, PISA_r_2009, PISA_sc_2009,
#               PISA_m_2012, PISA_r_2012, PISA_sc_2012,
#               PISA_m_2015, PISA_r_2015, PISA_sc_2015,
#               PISA_m_2018, PISA_r_2018, PISA_sc_2018,
#               PISA_m_2022, PISA_r_2022, PISA_sc_2022]

# for df in dataframes:
#     df['average'] = df['average'].astype(str).replace({'—': np.nan, '‡': np.nan, '†': np.nan})
#     df['average'] = df['average'].astype(float)
#     df['average'] = df['average'].round(2)

#     df['standard_error'] = df['standard_error'].astype(str).replace({'—': np.nan, '‡': np.nan, '†': np.nan})
#     df['standard_error'] = df['standard_error'].astype(float)
#     df['standard_error'] = df['standard_error'].round(2)

In [ ]:
dataframes = [PISA_m_2000, PISA_r_2000, PISA_sc_2000,
              PISA_m_2003, PISA_r_2003, PISA_sc_2003,
              PISA_m_2006, PISA_r_2006, PISA_sc_2006,
              PISA_m_2009, PISA_r_2009, PISA_sc_2009,
              PISA_m_2012, PISA_r_2012, PISA_sc_2012,
              PISA_m_2015, PISA_r_2015, PISA_sc_2015,
              PISA_m_2018, PISA_r_2018, PISA_sc_2018,
              PISA_m_2022, PISA_r_2022, PISA_sc_2022]

for df in dataframes:
    df['average'] = df['average'].apply(pd.to_numeric, errors='coerce')
    df['average'] = df['average'].round(2)

    df['standard_error'] = df['standard_error'].apply(pd.to_numeric, errors='coerce')
    df['standard_error'] = df['standard_error'].round(2)

In [ ]:
df['average'] = df['average'].astype('str')
df['average'] = df['average'].str.replace('-', '0')
df['average'] = df['average'].astype

In [ ]:
PISA_m_2006.sample(5)

---